In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy

py_file_location = './drive/My Drive/PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

In [ ]:
LABEL = 'trend'
MODEL = 'logr'

In [ ]:
curated_data = pd.read_csv('./drive/My Drive/LAB/Hackathon/data/curated_data.csv')
curated_data = curated_data.drop(columns=['closePrice', 'closePriceNorm', 'log_ret_normalised_by_day', 'log_ret', 'stock'])
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [ ]:
curated_data

In [ ]:
date = [datetime.datetime(year=2023, month=1, day = 1) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [ ]:
function_map = ['min', 'max', 'std', 'mean', 'slope', 'skew', 'abs_diff_mean', 'diff_std', 'sum']

for i in [3, 5, 10, 20]:
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'log_ret_normalised_by_day_lag_{j}', f'closePriceNorm_lag_{j}', f'closePrice_lag_{j}'], axis = 1)

    for j in [3, 5, 10, 20]:
        if j > i:

            columns = []
            for col in ['log_ret', 'closePrice', 'closePriceNorm', 'log_ret_normalised_by_day']:
                for func in function_map:
                    columns.append(f'{col}_{func}_{j}')
            tmp_curated_data = tmp_curated_data.drop(columns=columns, axis = 1)

    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]
    test = tmp_curated_data[(tmp_curated_data['date'] >= test_date_start)]

    jiaocheng = JiaoCheng.JiaoCheng()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']

    test_x = test.drop(columns=['date', 'trend'], axis = 1)
    test_y = test['trend']

    with open(f'./drive/My Drive/LAB/Hackathon/models/NingXiang_output_{LABEL}_lag_{i}.pickle', 'rb') as f:
        ningxiang_output = pickle.load(f)

    ningxiang_output = list(ningxiang_output.items())
    ningxiang_output = dict(ningxiang_output)

    jiaocheng = JiaoCheng.JiaoCheng()

    from sklearn.linear_model import LogisticRegression as LogR

    jiaocheng.read_in_model(LogR, 'Classification')

    jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

    del train_x
    del train_y
    del val_x
    del val_y

    parameter_choices = {
        'C': (0.001, 0.01, 0.1, 1, 10),
    }

    jiaocheng.set_hyperparameters(parameter_choices)

    jiaocheng.set_non_tuneable_hyperparameters({'max_iter': 10000, 'penalty': 'l1', 'random_state': 19260817, 'n_jobs': -1, 'solver': 'liblinear'})

    jiaocheng.set_features(ningxiang_output)

    jiaocheng.set_tuning_order(['features', 'C'])

    jiaocheng.set_hyperparameter_default_values({
        'features': 0,
        'C': 1,}
    )

    try:
        jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    except:
        pass

    jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tmp_models/jiaocheng_{MODEL}_{LABEL}_lag_{i}.pickle')

    jiaocheng.tune()

    jiaocheng.view_best_combo_and_score()

    print('\n\n')